In [ ]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install nltk

In [8]:
import pandas as pd
import numpy as np

In [9]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
credits.head(1)['crew'].values

In [ ]:
movies.shape

In [14]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.head(1)

In [ ]:
movies['original_language'].value_counts()

# as the output is highly unstable most of the bias is to english language.

In [19]:
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [22]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [26]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# The above list of dictionaries is converted only a simple list ['Action','Adventure','Fantasy',....]

In [27]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L    

In [ ]:
#ast[ABSTRACT SYNTAX TREES] is used to convert list of strings to list of literals. 
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
#convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [30]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head(1)

In [32]:
movies['keywords'] = movies['keywords'].apply(convert)

In [33]:
import ast
def convert3(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break    
    return L    

In [34]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['cast']

In [36]:
import ast
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L    

In [37]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies['crew']

In [39]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [43]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [45]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head(1)

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags'][0]

In [51]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [52]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][1]

VECTORIZATION


In [63]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [64]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [57]:
#STEMMING CONCEPT
#['loved','loving','love']
#['love','love','love']

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [73]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0].shape

In [82]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')